In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re # Regular expressions om getallen te extraheren

# --- Configuratie (Aangepast aan Nijmegen voorbeeld HTML) ---
BASE_URL = "https://www.zorgkaartnederland.nl"
# START_PATH voor Nijmegen (pas aan als je Arnhem wilt)
START_PATH = "/verpleeghuis-en-verzorgingshuis/arnhem"
PARAMS = "?d=25" # Behoud de afstandsparameter
MAX_PAGES = 10 # Je kunt dit aanpassen indien nodig
# Output bestandsnaam voor Nijmegen (pas aan als je Arnhem wilt)
OUTPUT_FILE = "zorginstellingen_arnhem.json"
# Regio voor Nijmegen (pas aan als je Arnhem wilt)
REGION = "Arnhem"

# Headers om een browser na te bootsen
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# --- CSS Selectors (!!! BIJGEWERKT OP BASIS VAN VERSTREKTE HTML !!!) ---
# Deze zouden nu beter moeten kloppen, maar controleer altijd bij wijzigingen op de site.
selectors = {
    "result_item": "div.filter-result",              # Container voor elke instelling
    "name": "a.filter-result__name",                 # Link met de naam van de instelling
    "places": "div.filter-result__places",           # Div die zowel afstand als plaats bevat
    "distance": "div.filter-result__places span.badge", # De span met de afstand (binnen places div)
    "rating_score": "div.filter-result__score",      # De div met het cijfer (kan <sup> bevatten)
    "rating_count_container": "div.filter-result__body__right p" # De <p> waar het aantal waarderingen in staat
}
# ------------------------------------------------------------------------------------

def parse_rating_count(text):
    """Probeert het aantal waarderingen uit een tekst te halen."""
    if not text:
        return None
    # Zoekt naar het eerste getal in de string
    match = re.search(r'\d+', text)
    if match:
        return int(match.group(0))
    return None

def scrape_zorgkaart():
    all_data = []
    session = requests.Session()
    session.headers.update(HEADERS)

    for page_num in range(1, MAX_PAGES + 1):
        print(f"Scraping pagina {page_num} voor {REGION}...")

        if page_num == 1:
            page_path = START_PATH + PARAMS
        else:
            page_path = f"{START_PATH}/pagina{page_num}{PARAMS}"

        current_url = BASE_URL + page_path
        print(f"Fetching: {current_url}")

        try:
            response = session.get(current_url, timeout=15)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            results = soup.select(selectors["result_item"])

            if not results:
                print(f"Geen resultaten gevonden op pagina {page_num}. Stoppen.")
                break

            print(f"Aantal resultaten gevonden op pagina {page_num}: {len(results)}")

            for item in results:
                data = {"Regio": REGION}

                # --- Gegevens extractie met bijgewerkte selectors ---
                name_element = item.select_one(selectors["name"])
                data["Naam"] = name_element.text.strip() if name_element else None
                data["Url"] = BASE_URL + name_element['href'] if name_element and name_element.has_attr('href') else None

                # Afstand eerst apart extraheren
                distance_element = item.select_one(selectors["distance"])
                distance_text = distance_element.text.strip() if distance_element else None
                data["Afstand"] = distance_text

                # Plaatsnaam extraheren en opschonen (verwijder afstand)
                places_element = item.select_one(selectors["places"])
                if places_element:
                    full_places_text = places_element.text.strip()
                    # Verwijder de afstandstekst (indien gevonden) uit de volledige tekst
                    location_text = full_places_text.replace(distance_text, '').strip() if distance_text else full_places_text
                    data["Locatie"] = location_text
                else:
                    data["Locatie"] = None

                # Cijfer (score)
                rating_score_element = item.select_one(selectors["rating_score"])
                if rating_score_element:
                    rating_text = rating_score_element.get_text(strip=True).replace(',', '.')
                    try:
                        # -----> DEZE CONVERSIE NAAR float() IS CRUCIAAL <-----
                        data["Cijfer"] = float(rating_text)
                    except ValueError:
                        print(f"Waarschuwing: Kon cijfer '{rating_text}' niet converteren naar getal voor {data.get('Naam')}.")
                        data["Cijfer"] = None # Wordt null in JSON
                else:
                    data["Cijfer"] = None # Wordt null in JSON
                # Aantal waarderingen
                rating_count_container = item.select_one(selectors["rating_count_container"])
                rating_count_text = rating_count_container.text.strip() if rating_count_container else None
                data["Aantal Waarderingen"] = parse_rating_count(rating_count_text)


                if data["Naam"]:
                    all_data.append(data)
                else:
                    print(f"Waarschuwing: Item overgeslagen op pagina {page_num} wegens ontbrekende naam.")


            time.sleep(1) # Pauze tussen pagina's

        except requests.exceptions.RequestException as e:
            print(f"Fout bij ophalen {current_url}: {e}")
            break
        except Exception as e:
            print(f"Onverwachte fout tijdens parsen pagina {page_num}: {e}")
            continue # Probeer eventueel volgende pagina

    return all_data

if __name__ == "__main__":
    print(f"Starten met scrapen van ZorgkaartNederland voor regio: {REGION}...")
    try:
        import requests
        from bs4 import BeautifulSoup
    except ImportError:
        print("FOUT: Libraries 'requests' en/of 'beautifulsoup4' niet geïnstalleerd.")
        print("Installeer ze met: pip install requests beautifulsoup4")
        exit()

    scraped_data = scrape_zorgkaart()
    print(f"Scrapen voltooid. Totaal {len(scraped_data)} instellingen gevonden voor {REGION}.")

    if scraped_data:
        try:
            with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
                json.dump(scraped_data, f, ensure_ascii=False, indent=4)
            print(f"Data succesvol opgeslagen in {OUTPUT_FILE}")
        except IOError as e:
            print(f"Fout bij schrijven naar bestand {OUTPUT_FILE}: {e}")
        except Exception as e:
            print(f"Onverwachte fout bij opslaan JSON: {e}")

        # print("\n--- Voorbeeld eerste resultaat ---")
        # if scraped_data:
        #     print(json.dumps(scraped_data[0], ensure_ascii=False, indent=4))

    else:
        print("Geen data gescraped. Controleer de selectors, URL-instellingen en website beschikbaarheid.")

Starten met scrapen van ZorgkaartNederland voor regio: Arnhem...
Scraping pagina 1 voor Arnhem...
Fetching: https://www.zorgkaartnederland.nl/verpleeghuis-en-verzorgingshuis/arnhem?d=25
Aantal resultaten gevonden op pagina 1: 20
Scraping pagina 2 voor Arnhem...
Fetching: https://www.zorgkaartnederland.nl/verpleeghuis-en-verzorgingshuis/arnhem/pagina2?d=25
Aantal resultaten gevonden op pagina 2: 20
Scraping pagina 3 voor Arnhem...
Fetching: https://www.zorgkaartnederland.nl/verpleeghuis-en-verzorgingshuis/arnhem/pagina3?d=25
Aantal resultaten gevonden op pagina 3: 20
Scraping pagina 4 voor Arnhem...
Fetching: https://www.zorgkaartnederland.nl/verpleeghuis-en-verzorgingshuis/arnhem/pagina4?d=25
Aantal resultaten gevonden op pagina 4: 20
Scraping pagina 5 voor Arnhem...
Fetching: https://www.zorgkaartnederland.nl/verpleeghuis-en-verzorgingshuis/arnhem/pagina5?d=25
Aantal resultaten gevonden op pagina 5: 20
Scraping pagina 6 voor Arnhem...
Fetching: https://www.zorgkaartnederland.nl/verple